# RAG - How to query

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_HTTP_URL = os.getenv("WEAVIATE_HTTP_URL")
WEAVIATE_GRPC_URL = os.getenv("WEAVIATE_GRPC_URL")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_BASE_URL = os.getenv("AZURE_BASE_URL")
AZURE_RESOURCE_NAME = os.getenv("AZURE_RESOURCE_NAME")

USERNAME = os.getenv("USERNAME")
WIKI_NAME = f"{USERNAME}_wiki"

print(WIKI_NAME)

## Connect to Weaviate

In [ ]:
import weaviate
from weaviate.classes.init import AdditionalConfig, Timeout

client = weaviate.connect_to_custom(
    http_host=WEAVIATE_HTTP_URL,
    http_port=80,
    http_secure=False,
    grpc_host=WEAVIATE_GRPC_URL,
    grpc_port=50051,
    grpc_secure=False,

    headers = {
        "X-Azure-Api-Key": AZURE_OPENAI_API_KEY
    },

    additional_config=AdditionalConfig(
        timeout=Timeout(query=60)  # Values in seconds
    )
)

client.is_ready()

### Start with (R) - Retrieval

In [ ]:
wiki = client.collections.get(WIKI_NAME)

response = wiki.query.near_text(
    query="How do planes fly",
    limit=5,
    return_properties=["text", "title"]
)

for item in response.objects:
    print(item.properties)

### Add (AG) - augmented generation - to make full RAG

#### Single Prompt

> Generate a response per **retrieved** object.

In [ ]:
# Let's add some colour to our lives :)
BLUE   = "\033[94m"
PURPLE = "\033[95m"
RESET  = "\033[0"

In [ ]:
from weaviate.classes.generate import GenerativeConfig

wiki = client.collections.get(WIKI_NAME)

response = wiki.generate.near_text(
    query="How do planes fly",
    # auto_limit=1,
    limit=5,

    # TODO: add Azure GenerativeConfig with deployment_id="gpt-4o-mini-0718", resource_name, and base_url.
    # generative_provider=

    # TODO: add a single prompt "Explain what this is about? {text}"
    # single_prompt=
)

# NOTE: generated responses are included with each object
for item in response.objects:
    print(f"{BLUE}=== Source ===")
    print(item.properties)

    print(f"{PURPLE}=== Generated Response ===")
    # TODO: print the generative.text object
    # print(item.)

    print("\n")

#### Grouped Task

> Generate one response based on all **retrieved** objects.

In [ ]:
response = wiki.generate.near_text(
    query="How do planes fly",
    # auto_limit=1,
    limit=5,

    generative_provider=GenerativeConfig.openai_azure(
        deployment_id="gpt-4o-mini-0718",   # select your generative model
        resource_name=AZURE_RESOURCE_NAME,
        base_url=AZURE_BASE_URL 
    ),
    
    # TODO: add a grouped task "Explain, how do planes fly? Please only use the provided content."
    # grouped_task=
)

print(f"{PURPLE}=== Generated Response ===")
# NOTE: group task response is at response.generative.text
# TODO: print the generated text
# print(response.)

print(f"{BLUE}=== Source ===")
for item in response.objects:
    print(item.properties)

#### Specify which properties to use for grouped task

In [ ]:
response = wiki.generate.near_text(
    query="How do planes fly",
    auto_limit=1,

    generative_provider=GenerativeConfig.openai_azure(
        deployment_id="gpt-4o-mini-0718",   # select your generative model
        resource_name=AZURE_RESOURCE_NAME,
        base_url=AZURE_BASE_URL
    ),
    
    grouped_task="Explain, how do planes fly? Please only use the provided content.",
    # TODO: add grouped properties to only use "text" and "title" 
    # grouped_properties=[]
)

print("=== Generated Response ===")
print(response.generative.text)

print("=== Source ===")
for item in response.objects:
    print(item.properties)

## Set default Generative model

In [ ]:
from weaviate.classes.config import Reconfigure

wiki = client.collections.get(WIKI_NAME)

wiki.config.update(
    #TODO: set generative model
    # generative_config=Reconfigure.Generative.
)

>Try generative query without providing the model

In [ ]:

response = wiki.generate.near_text(
    query="What african animals do we have info on. Please only list those provided in here.",
    auto_limit=1,
    
    grouped_task="Explain, how do planes fly? Please only use the provided content.",
)

print(f"{PURPLE}=== Generated Response ===")
print(response.generative.text)

print(f"{BLUE}=== Source ===")
for item in response.objects:
    print(item.properties)

## Close the client

In [ ]:
client.close()